<a href="https://colab.research.google.com/github/Hongyongmin/Edwith-Pytorch/blob/main/10_2_Mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
device

'cuda'

In [5]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train=True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [7]:
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding =1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 2, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

    
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out

In [8]:
model = CNN().to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
  avg_cost =0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch:: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch::    1] cost = 0.273991019
[Epoch::    2] cost = 0.0730143487
[Epoch::    3] cost = 0.052894447
[Epoch::    4] cost = 0.0435905233
[Epoch::    5] cost = 0.0372963771
[Epoch::    6] cost = 0.0329324678
[Epoch::    7] cost = 0.0279795136
[Epoch::    8] cost = 0.0250027552
[Epoch::    9] cost = 0.021250179
[Epoch::   10] cost = 0.0189303476
[Epoch::   11] cost = 0.0172208697
[Epoch::   12] cost = 0.0141859725
[Epoch::   13] cost = 0.0116033768
[Epoch::   14] cost = 0.0109210014
[Epoch::   15] cost = 0.00967126153
Learning Finished!


In [11]:
with torch.no_grad():

  X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1 ) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.983299970626831


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


좀더 딥하게 만들어 보면 아래임

In [12]:
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding =1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 2, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size=3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride = 2, padding = 1))
    self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p = 1 - self.keep_prob))
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

    
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0),-1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [13]:
model = CNN().to(device)


In [14]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
device

'cuda'

In [16]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.219943002
[Epoch:    2] cost = 0.0536558218
[Epoch:    3] cost = 0.0375116989
[Epoch:    4] cost = 0.0293143652
[Epoch:    5] cost = 0.0239973031
[Epoch:    6] cost = 0.0210262723
[Epoch:    7] cost = 0.0180218685
[Epoch:    8] cost = 0.0148797771
[Epoch:    9] cost = 0.0139938882
[Epoch:   10] cost = 0.0123479469
[Epoch:   11] cost = 0.0108595826
[Epoch:   12] cost = 0.0108660059
[Epoch:   13] cost = 0.00815076567
[Epoch:   14] cost = 0.00786682218
[Epoch:   15] cost = 0.00894461852
Learning Finished!


In [17]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.989799976348877


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
